In [66]:
import theano
import theano.tensor as T
import theano.tensor.nnet as nnet
import numpy as np
import time

import cv2 
import numpy as np
import math
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import signal
import time
from math import sqrt
from skimage import transform
import skimage 


In [ ]:
training_data = readTrainingdata('./training/train/img0_train0')

In [240]:
import random

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
#             print delta_nabla_b[0][0][0]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
#             if nabla_w[0][0][0] > 0:
#                 print nabla_w[0][0][0]
#                 print self.weights[0][0][0]-(eta * nabla_w[0][0][0])
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        print activations[-1]
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        print delta
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def update(self, x, y):
        
        nabla_b, nabla_w = self.backprop(x, y)        
        self.weights = [w - nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - nb for b, nb in zip(self.biases, nabla_b)]
         
    
    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        """set a limit"""
        match_limit = 0.1
        output = sum(int(abs(x - y) < match_limit) for (x, y) in test_results)
#         print test_results[0][0] - test_results[0][1]
#         print test_results[10][0] - test_results[10][1]
#         print test_results[20][0] - test_results[20][1]        
        return output

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))
    


In [241]:
def readTrainingdata(filename):
    
    trainingData = []
    theFile = open(filename, 'r')
    line = theFile.readline()
    while line != None and line != "":
        values = line.split()
        y = np.array([float(values[0])])        
        x = np.asarray(values[1:], dtype=float).reshape(29,1)
        trainingData.append((x, y))
        line = theFile.readline()
    
    return trainingData

def getImgFromNet(net, img):
    
    width, height = img.shape
    filtered = np.zeros((width, height))

    radius = 3
    kernel = circle(radius)
    denominator = kernel.sum()
    img_force_int = img.astype('float')
    for x in range(radius, width-radius):
        for y in range(radius, height-radius):
            crop = img_force_int[x - radius:x + radius + 1, y - radius:y + radius + 1]
            croped_crop = crop[kernel == 1].reshape(29,1)
            output = net.feedforward(croped_crop)
            filtered[x, y] = output
    
    return filtered

In [242]:
test_data_batch = []
for i in range(0,5):    
    test_data_batch.append(training_data[random.randint(0,len(training_data)-1)])
    
net = Network([29, 50, 50, 1])

random.shuffle(training_data)
net.SGD(training_data[0:800], epochs=1, mini_batch_size=100, eta=10, test_data=test_data_batch)

[[ 0.45868713]]
[[-0.10348069]]
[[ 0.99628413]]
[[ 0.00511801]]
[[ 0.85739264]]
[[-0.03893754]]
[[ 0.34168751]]
[[-0.18493063]]
[[ 0.9431111]]
[[-0.0182889]]
[[ 0.28216104]]
[[ 0.00395059]]
[[ 0.77864048]]
[[ 0.35816647]]
[[ 0.99018047]]
[[ 0.00777477]]
[[ 0.70234752]]
[[ 0.39335264]]
[[ 0.95179735]]
[[ 0.04319384]]
[[ 0.96151744]]
[[-0.01015353]]
[[ 0.81841693]]
[[-0.06561746]]
[[ 0.7190948]]
[[-0.0752728]]
[[ 0.87801436]]
[[ 0.01571737]]
[[ 0.95105312]]
[[-0.01102373]]
[[ 0.70865069]]
[[-0.09397876]]
[[ 0.59851851]]
[[ 0.52534876]]
[[ 0.7673443]]
[[ 0.07508459]]
[[ 0.92381577]]
[[ 0.17761053]]
[[ 0.6930401]]
[[-0.11804707]]
[[ 0.95083563]]
[[ 0.06868091]]
[[ 0.47579186]]
[[ 0.22398077]]
[[ 0.96885544]]
[[ 0.00245575]]
[[ 0.67004397]]
[[ 0.18835718]]
[[ 0.73411823]]
[[ 0.43209553]]
[[ 0.46143304]]
[[-0.15962453]]
[[ 0.97561144]]
[[-0.00647974]]
[[ 0.46768428]]
[[-0.00280176]]
[[ 0.65079951]]
[[ 0.12643864]]
[[ 0.52295377]]
[[-0.18386234]]
[[ 0.78374883]]
[[ 0.2145808]]
[[ 0.59270116]]

In [244]:


weightsBefore = [net.weights[2][0][0:5], net.weights[1][0][0:5], net.weights[0][0][0:5]]
# print weightsBefore
# print training_data[0][0][0]
random.shuffle(training_data)
# print training_data[0][0][0]
net.SGD(training_data[0:800], epochs=1, mini_batch_size=10, eta=1000, test_data=test_data_batch)

weightsAfter = [net.weights[2][0][0:5], net.weights[1][0][0:5], net.weights[0][0][0:5]]
# print weightsAfter
for x, y in zip(weightsAfter, weightsBefore):
    print x - y

[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[ 0.]]
[[-0.]]
[[ 0.]]


In [189]:
weightsBefore = [net.weights[2][0][0:5], net.weights[1][0][0:5], net.weights[0][0][0:5]]
# print weightsBefore
# random.shuffle(training_data)
net.SGD(training_data[0:800], epochs=1, mini_batch_size=100, eta=100000000000000000000, test_data=test_data_batch)

weightsAfter = [net.weights[2][0][0:5], net.weights[1][0][0:5], net.weights[0][0][0:5]]
# print weightsAfter
for x, y in zip(weightsAfter, weightsBefore):
    print x - y

Epoch 0: 0 / 5
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
